In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import json
# import xlrd
pd.set_option('display.max_columns', None)

## Preprocess the data (agian...)

1. read the new MAU data and fill all the nan with 0.0
2. join it with previous ready-to-use dataframe
3. calculate the AR columns for 12 workloads
4. convert NaN to 0.0 and INF to a real number
5. clip the AR data to (0.0 to 1.0)
6. seperate mature and young tenants
7. create related configure file.

In [21]:
df_u = pd.read_csv('IDEAS_newMAU-2019-06.csv')

In [22]:
df_u.head()

,TenantId,AU_eslt,AU_exchange,AU_excel,AU_oatp,AU_od4b,AU_odsp,AU_officeclient,AU_onenote,AU_outlook,AU_powerpoint,AU_sharepoint,AU_skype,AU_teams,AU_word
0,00000000-0000-0000-0000-000000000000,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,9.0,NaN,NaN,NaN,NaN,NaN
1,00000051-56F4-4474-B2C0-68559D76E254,1.0,1.0,1.0,NaN,NaN,NaN,1.0,NaN,1.0,1.0,NaN,NaN,NaN,1.0
2,000001F9-6BCA-4027-A3A8-243B62FB52DE,1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,0000072E-8DC0-43CB-8FEE-9B366176B190,1.0,1.0,2.0,NaN,1.0,1.0,2.0,NaN,2.0,NaN,NaN,NaN,NaN,2.0
4,00000875-0649-4EF7-BB1C-A7C090441EE3,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN


In [23]:
df_u = df_u.iloc[1:, :]

In [24]:
df_u.drop(['AU_oatp', 'AU_odsp'], axis=1, inplace=True)

In [25]:
df_u = df_u.fillna(0.0)

In [26]:
df_u.shape

(5207078, 13)

In [27]:
df_u = df_u[['TenantId', 'AU_exchange',
 'AU_sharepoint',
 'AU_skype',
 'AU_teams',
 'AU_od4b',
 'AU_onenote',
 'AU_word',
 'AU_excel',
 'AU_powerpoint',
 'AU_outlook',
 'AU_eslt',
 'AU_officeclient']]

In [2]:
df_orig = pd.read_csv('TenantInfo-and-usage_shuffled_inf.csv')

/data/home/t-chepan/env/newlab/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (145,146) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_orig.shape

(1865326, 228)

In [5]:
df_orig['HasNonTrial'].describe()

count     1865326
unique          1
top          True
freq      1865326
Name: HasNonTrial, dtype: object

In [28]:
df = pd.merge(left=df_orig, right=df_u, on='TenantId',suffixes=('_06', '_06n'))

In [29]:
df.shape

(1863984, 240)

In [30]:
cols_ps = ['PaidEXOSeats','PaidSPOSeats','PaidSFBSeats','PaidTeamsSeats',
           'PaidOD4BSeats'] + ['PaidOfficeSeats']*5 +['PaidCount','PaidOfficeSeats']
cols_au = df.columns.tolist()[-12:]

In [31]:
cols_au

['AU_exchange_06n',
 'AU_sharepoint_06n',
 'AU_skype_06n',
 'AU_teams_06n',
 'AU_od4b_06n',
 'AU_onenote_06n',
 'AU_word_06n',
 'AU_excel_06n',
 'AU_powerpoint_06n',
 'AU_outlook_06n',
 'AU_eslt_06n',
 'AU_officeclient_06n']

In [32]:
cols_ps

['PaidEXOSeats',
 'PaidSPOSeats',
 'PaidSFBSeats',
 'PaidTeamsSeats',
 'PaidOD4BSeats',
 'PaidOfficeSeats',
 'PaidOfficeSeats',
 'PaidOfficeSeats',
 'PaidOfficeSeats',
 'PaidOfficeSeats',
 'PaidCount',
 'PaidOfficeSeats']

In [34]:
cols_ar = ['AR_exchange_06n',
 'AR_sharepoint_06n',
 'AR_skype_06n',
 'AR_teams_06n',
 'AR_od4b_06n',
 'AR_onenote_06n',
 'AR_word_06n',
 'AR_excel_06n',
 'AR_powerpoint_06n',
 'AR_outlook_06n',
 'AR_eslt_06n',
 'AR_officeclient_06n']


In [35]:
ar_list = list(zip(cols_ar, cols_au, cols_ps))

In [36]:
print(ar_list)

[('AR_exchange_06n', 'AU_exchange_06n', 'PaidEXOSeats'), ('AR_sharepoint_06n', 'AU_sharepoint_06n', 'PaidSPOSeats'), ('AR_skype_06n', 'AU_skype_06n', 'PaidSFBSeats'), ('AR_teams_06n', 'AU_teams_06n', 'PaidTeamsSeats'), ('AR_od4b_06n', 'AU_od4b_06n', 'PaidOD4BSeats'), ('AR_onenote_06n', 'AU_onenote_06n', 'PaidOfficeSeats'), ('AR_word_06n', 'AU_word_06n', 'PaidOfficeSeats'), ('AR_excel_06n', 'AU_excel_06n', 'PaidOfficeSeats'), ('AR_powerpoint_06n', 'AU_powerpoint_06n', 'PaidOfficeSeats'), ('AR_outlook_06n', 'AU_outlook_06n', 'PaidOfficeSeats'), ('AR_eslt_06n', 'AU_eslt_06n', 'PaidCount'), ('AR_officeclient_06n', 'AU_officeclient_06n', 'PaidOfficeSeats')]


In [37]:
def compute_ar(df, ar_list):
    for i in ar_list:
        df[i[0]] = df[i[1]] / df[i[2]]
    return df

In [38]:
df_ar = compute_ar(df, ar_list)

In [39]:
df_ar.iloc[:10, -12:]

,AR_exchange_06n,AR_sharepoint_06n,AR_skype_06n,AR_teams_06n,AR_od4b_06n,AR_onenote_06n,AR_word_06n,AR_excel_06n,AR_powerpoint_06n,AR_outlook_06n,AR_eslt_06n,AR_officeclient_06n
0,0.500000,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,inf,0.400000,inf
1,0.785714,0.010204,0.010204,0.010204,0.181818,inf,inf,inf,inf,inf,0.777778,inf
2,0.750000,0.000000,0.000000,0.000000,0.500000,inf,inf,inf,inf,inf,0.750000,inf
3,0.379310,0.034483,0.034483,0.137931,0.000000,NaN,inf,inf,inf,inf,0.482759,inf
4,0.916667,NaN,NaN,NaN,NaN,NaN,inf,inf,inf,inf,0.916667,inf
5,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inf,1.000000,inf
6,0.806630,0.085227,0.000000,0.131429,0.082418,inf,inf,inf,inf,inf,0.781915,inf
7,0.927711,NaN,NaN,NaN,NaN,inf,inf,inf,inf,inf,0.927711,inf
8,1.000000,0.000000,0.000000,0.000000,0.000000,NaN,inf,NaN,NaN,inf,1.000000,inf
9,1.000000,0.909091,0.000000,0.000000,0.363636,NaN,inf,inf,NaN,inf,1.000000,inf


In [40]:
# convert NaN to 0:
df_ar.fillna(0.0, inplace=True)

In [44]:
# convert inf to a numerical number:
conv_list = list(zip(cols_ar, cols_au))

t1 = time.time()
for c in conv_list[:]:
    df_ar[c[0]] = np.where(df_ar[c[0]] != np.inf, df_ar[c[0]], df_ar[c[1]]/df_ar['PaidCount'])
    print('Already complete coloumn {}'.format(c[0]))
t2 = time.time()
print(t2 - t1)

Already complete coloumn AR_exchange_06n
Already complete coloumn AR_sharepoint_06n
Already complete coloumn AR_skype_06n
Already complete coloumn AR_teams_06n
Already complete coloumn AR_od4b_06n
Already complete coloumn AR_onenote_06n
Already complete coloumn AR_word_06n
Already complete coloumn AR_excel_06n
Already complete coloumn AR_powerpoint_06n
Already complete coloumn AR_outlook_06n
Already complete coloumn AR_eslt_06n
Already complete coloumn AR_officeclient_06n
0.21718072891235352


In [45]:
df_ar.iloc[:10, -12:]

,AR_exchange_06n,AR_sharepoint_06n,AR_skype_06n,AR_teams_06n,AR_od4b_06n,AR_onenote_06n,AR_word_06n,AR_excel_06n,AR_powerpoint_06n,AR_outlook_06n,AR_eslt_06n,AR_officeclient_06n
0,0.500000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.400000,0.200000
1,0.785714,0.010204,0.010204,0.010204,0.181818,0.151515,0.444444,0.393939,0.373737,0.666667,0.777778,0.676768
2,0.750000,0.000000,0.000000,0.000000,0.500000,0.250000,0.500000,0.500000,0.250000,0.500000,0.750000,0.750000
3,0.379310,0.034483,0.034483,0.137931,0.000000,0.000000,0.241379,0.241379,0.137931,0.275862,0.482759,0.275862
4,0.916667,0.000000,0.000000,0.000000,0.000000,0.000000,0.416667,0.083333,0.083333,0.750000,0.916667,0.750000
5,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,1.000000,0.083333
6,0.806630,0.085227,0.000000,0.131429,0.082418,0.015957,0.281915,0.255319,0.132979,0.388298,0.781915,0.398936
7,0.927711,0.000000,0.000000,0.000000,0.000000,0.004016,0.092369,0.196787,0.008032,0.586345,0.927711,0.586345
8,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.666667,0.000000,0.000000,1.000000,1.000000,1.000000
9,1.000000,0.909091,0.000000,0.000000,0.363636,0.000000,0.818182,0.818182,0.000000,0.909091,1.000000,0.909091


In [46]:
df_ar.shape

(1863984, 252)

In [51]:
drop_cols = ['AR_exchange_04',
 'AR_sharepoint_04',
 'AR_skype_04',
 'AR_teams_04',
 'AR_od4b_04',
 'AR_onenote_04',
 'AR_word_04',
 'AR_excel_04',
 'AR_powerpoint_04',
 'AR_outlook_04',
 'AR_eslt_04',
 'AR_officeclient_04'] + ['AR_exchange_05',
 'AR_sharepoint_05',
 'AR_skype_05',
 'AR_teams_05',
 'AR_od4b_05',
 'AR_onenote_05',
 'AR_word_05',
 'AR_excel_05',
 'AR_powerpoint_05',
 'AR_outlook_05',
 'AR_eslt_05',
 'AR_officeclient_05'] + ['AR_exchange_06',
 'AR_sharepoint_06',
 'AR_skype_06',
 'AR_teams_06',
 'AR_od4b_06',
 'AR_onenote_06',
 'AR_word_06',
 'AR_excel_06',
 'AR_powerpoint_06',
 'AR_outlook_06',
 'AR_eslt_06',
 'AR_officeclient_06'] + ['AU_exchange_04',
 'AU_sharepoint_04',
 'AU_skype_04',
 'AU_teams_04',
 'AU_od4b_04',
 'AU_onenote_04',
 'AU_word_04',
 'AU_excel_04',
 'AU_powerpoint_04',
 'AU_outlook_04',
 'AU_eslt_04',
 'AU_officeclient_04',
 'AU_exchange_05',
 'AU_sharepoint_05',
 'AU_skype_05',
 'AU_teams_05',
 'AU_od4b_05',
 'AU_onenote_05',
 'AU_word_05',
 'AU_excel_05',
 'AU_powerpoint_05',
 'AU_outlook_05',
 'AU_eslt_05',
 'AU_officeclient_05',
 'AU_exchange_06',
 'AU_sharepoint_06',
 'AU_skype_06',
 'AU_teams_06',
 'AU_od4b_06',
 'AU_onenote_06',
 'AU_word_06',
 'AU_excel_06',
 'AU_powerpoint_06',
 'AU_outlook_06',
 'AU_eslt_06',
 'AU_officeclient_06'] +['AU_oatp_04','AU_odsp_04','AU_oatp_05','AU_odsp_05','AU_oatp','AU_odsp']

In [52]:
len(drop_cols)

78

In [54]:
df_ar = df_ar.drop(columns=drop_cols)

In [55]:
df_ar.shape

(1863984, 174)

## Seperate mature and young tenants

In [58]:
def clip_wl_ar_cols(df, cols_ar):    

    wl_arr = df.loc[:, cols_ar].to_numpy()
    wl_clip = np.clip(wl_arr, 0.0, 1.0)

    df.loc[:, cols_ar] = wl_clip
    
    return df

In [59]:
df_clip = clip_wl_ar_cols(df_ar, cols_ar)

In [60]:
df_clip.to_csv('TenantInfo-and-usage_wNewMAU_clip.csv', index=None)

In [61]:
def split_mature_and_young(df, age=360):
    df_mature = df.loc[df['Age'] >= age]
    df_young = df.loc[df['Age'] < age]
    
    df_mature = df_mature.drop('Age', axis=1)
    df_young = df_young.drop('Age', axis=1)
    
    print('full dataset shape is {}'.format(df.shape))
    print('df_young shape is {}'.format(df_young.shape))
    return df_mature, df_young

In [62]:
df_c_mature, df_c_young = split_mature_and_young(df_clip) 

full dataset shape is (1863984, 174)
df_young shape is (478619, 173)


In [63]:
df_c_young = df_c_young.sample(frac=1.0)
df_c_young.to_csv('TenantInfo-and-usage_wNewMAU_young_clip_shuffled.csv', index=None)

In [64]:
df_c_mature = df_c_mature.sample(frac=1.0)
df_c_mature.to_csv('TenantInfo-and-usage_wNewMAU_mature_clip_shuffled.csv', index=None)

## Create the related configure file

In [56]:
def get_input_details(df, input_cols, has_text_features=False):
    
    df_input = df.loc[:, input_cols]
    print(len(input_cols))
    
    ## you can also hard code the different types of input features
#     df_float = df_input.select_dtypes(include=['float'])
#     input_float_cols = df_float.columns.tolist()
    input_float_cols = []
    
    
    input_int_cols = [
        'AllupSeats',
        'EXOSubscriptionsCount',
        'OD4BSubscriptionsCount',
        'SfBSubscriptionsCount',
        'TeamsSubscriptionsCount',
        'PaidCount',
        'ProjectSubscriptionsCount',
        'SPOSubscriptionsCount',
        'ActivatedSubscriptionTotalCount',
        'VisioSubscriptionsCount',
        'TrialSubscriptionsCount',
        'NonTrialSubscriptionsCount',
        'EXOEnabledUsers',
        'SPOEnabledUsers',
        'OD4BEnabledUsers',
        'SFBEnabledUsers',
        'TeamEnabledUsers',
        'YammerEnabledUsers',
        'PPDEnabledUsers',
        'KaizalaEnabledUsers',
        'AADPEnabledUsers',
        'AIPEnabledUsers',
        'AATPEnabledUsers',
        'IntuneEnabledUsers',
        'MCASEnabledUsers',
        'WDATPEnabledUsers',
        'AudioConferenceEnabledUsers',
        'PhoneSystemEnabledUsers',
        'EdiscoveryEnabledUsers',
        'ComplianceEnabledUsers ',
        'ThreatIntelligenceEnabledUsers',
        'CustomerLockboxEnabledUsers',
        'OATPEnabledUsers',
        'AADPP2EnabledUsers',
        'AIPP2EnabledUsers',
        'WindowsEnabledUsers ',
        'O365CASEnabledUsers',
        'CASDiscoveryEnabledUsers',
        'PAMEnabledUsers',
        'O365EnabledUsers',
        'EMSEnabledUsers',
        'M365EnabledUsers',
        'O365E5EnabledUsers',
        'EMSE5EnabledUsers',
        'M365E5EnabledUsers',
        'TotalUsers',
        'PaidEXOSeats',
        'PaidSPOSeats',
        'PaidOD4BSeats',
        'PaidYammerSeats',
        'PaidTeamsSeats',
        'PaidSFBSeats',
        'PaidKaizalaSeats',
        'PaidProplusSeats',
        'PaidAADPSeats',
        'PaidAIPSeats',
        'PaidAATPSeats',
        'PaidIntuneSeats',
        'PaidMCASSeats',
        'PaidWDATPSeats',
        'PaidPhoneSystemSeats',
        'PaidAIPP2Seats',
        'PaidAudioConferenceSeats',
        'PaidOATPSeats',
        'PaidAADPP2Seats',
        'PaidWindowsSeats',
        'PaidO365CASSeats',
        'PaidCASDiscoverySeats',
        'PaidPAMSeats',
        'PaidPowerBISeats',
        'PaidPowerBIPremiumSeats',
        'PaidPowerBIProSeats',
        'PaidEMSSeats',
        'PaidM365Seats',
        'PaidOfficeSeats'
        ]
    
    input_datetime_cols =  ['CreatedDate', 'CreateDateOfFirstSubscription','FirstPaidEXOStartDate',
       'FirstPaidSPOStartDate', 'FirstPaidOD4BStartDate',
       'FirstPaidSfBStartDate', 'FirstPaidTeamsStartDate', 'FirstPaidProPlusStartDate']
    
    input_categotical_cols = ['CountryCode', 'Languange', 'TopParents_Industry']
    
    df_bool = df_input.select_dtypes(include=['bool'])
    input_bool_cols = df_bool.columns.tolist()
    
    if has_text_features == True:
        input_text_cols = ['Comment', 'Theme']
    else:
        input_text_cols = []
    
    input_drop_cols = ['TenantId', 'FirstPaidYammerStartDate', 'FirstPaidAADPStartDate', 'FirstPaidAIPStartDate',
       'FirstPaidAATPStartDate', 'FirstPaidIntuneStartDate',
       'FirstPaidMCASStartDate', 'FirstPaidO365E5SkuStartDate',
       'FirstPaidM365E5SkuStartDate', 'FirstPaidEMSE5SkuStartDate',
       'DataCenterInstance', 'DataCenterModel',
       'SignupLocationInfo_Country', 'SignupLocationInfo_CountryCode',
       'SignupLocationInfo_Region', 'TopParents_AreaName',
       'TopParents_CountryCode', 'TopParents_BigAreaName', 
       'TopParents_RegionName', 'OrgNameEnteredInCommerce',
       'TopParents_SegmentGroup', 'TopParents_SubRegionName',
       'TopParents_VerticalName'
                      ]
    
    n = len(input_text_cols) + len(input_float_cols) + len(input_int_cols) + len(
        input_categotical_cols) + len(input_bool_cols) + len(input_datetime_cols) + len(input_drop_cols)
    
    print(n)
    print(len(input_cols))
    
    assert len(input_cols) ==  n
    
    return (input_text_cols, input_float_cols, input_int_cols, input_categotical_cols,
            input_datetime_cols, input_bool_cols, input_drop_cols)

    

In [57]:
def separate_input_output_cols(df, has_text_features=False):
    cols_name = pd.Series(data=df.columns)

    ar_beg_col_index = cols_name[cols_name == 'AR_exchange_06n'].index[0]
    ar_end_col_index = cols_name[cols_name == 'AR_officeclient_06n'].index[0]

    au_beg_col_index = cols_name[cols_name == 'AU_exchange_06n'].index[0]
    au_end_col_index = cols_name[cols_name == 'AU_officeclient_06n'].index[0]

    wl_AR_cols = cols_name[ar_beg_col_index:ar_end_col_index+1].tolist()
    wl_AU_cols = cols_name[au_beg_col_index:au_end_col_index+1].tolist()
    
    all_cols = cols_name.tolist()
    
    output_cols = wl_AR_cols
    
    # use profile info only, exclude the usage of previous months (exclude AR and AU)
    input_cols = [x for x in all_cols if x not in (wl_AR_cols + wl_AU_cols)]
    
    (input_text_cols, input_float_cols, input_int_cols, input_categotical_cols, input_datetime_cols,
     input_bool_cols, input_drop_cols) = get_input_details(df, input_cols, has_text_features=has_text_features)
        
        
    print('There are {} inputs and {} outputs'.format(len(input_cols), len(output_cols)))
    
    n = len(input_text_cols) + len(input_float_cols) + len(input_int_cols) + len(input_categotical_cols) + len(
        input_bool_cols) + len(input_datetime_cols) + len(input_drop_cols)
    
    print(len(input_float_cols))
    print(n)
    print(len(input_cols))
    
    
    assert len(input_cols) ==  n
    
    return (input_cols, output_cols, input_text_cols, input_float_cols, input_int_cols,
            input_categotical_cols, input_datetime_cols, input_bool_cols, input_drop_cols)

In [65]:
df = pd.read_csv('TenantInfo-and-usage_wNewMAU_young_clip_shuffled.csv')

In [66]:
(input_cols, output_cols, input_text_cols, input_float_cols, input_int_cols,
 input_categotical_cols, input_datetime_cols, input_bool_cols, input_drop_cols) = separate_input_output_cols(df)

## the basic configure file doesn't include the text features:

metadata = {}
metadata['input_features'] = input_cols
metadata['output_label'] = output_cols
metadata['input_text'] = input_text_cols
metadata['input_float'] = input_float_cols
metadata['input_int'] = input_int_cols
metadata['input_categorical'] = input_categotical_cols
metadata['input_datetime'] = input_datetime_cols
metadata['input_bool'] = input_bool_cols
metadata['input_drop'] = input_drop_cols



149
149
149
There are 149 inputs and 12 outputs
0
149
149


In [69]:
with open('configure_wNewMAU.json', 'w') as f:
    json.dump(metadata, f)